In [ ]:
import pyvista as pv

pv.set_jupyter_backend("static")

# Simple Example

Run a basic process-structure-property simulation using `Materialite`.

Import the core `Material` object, some models, and other required objects for model inputs.

In [ ]:
from materialite import Material, Order4SymmetricTensor
from materialite.models import GrainCoarseningModel
from materialite.models.small_strain_fft import SmallStrainFFT, Elastic, LoadSchedule

Create a default material (generates a 16 x 16 x 16 point grid).

In [ ]:
material = Material()

Create a model for grain coarsening (`GrainCoarseningModel` is a Potts Monte Carlo-based model).

In [ ]:
grain_coarsening_model = GrainCoarseningModel(num_flip_attempts=10**5)

Run the grain coarsening model, and assign random orientations to the grains in the `Material` that is returned by the model.

In [ ]:
material = grain_coarsening_model(material)
material = material.assign_random_orientations(region_label="grain")

Plot the grain ID values.

In [ ]:
material.plot("grain")

Create a model that will simulate uniaxial tension (`SmallStrainFFT` is an FFT-based full-field mechanical simulation).

In [ ]:
stiffness = Order4SymmetricTensor.from_cubic_constants(C11=250, C12=150, C44=120)
constant_strain_rate = LoadSchedule.from_constant_uniaxial_strain_rate(
    magnitude=0.001, direction="x"
)
mechanical_model = SmallStrainFFT(
    load_schedule=constant_strain_rate, constitutive_model=Elastic(stiffness=stiffness)
)

Run the model.

In [ ]:
material = mechanical_model(material)

Plot the stress field in the loading direction.

In [ ]:
material.plot("stress", component=0)